In [1]:
# !pip install datasets evaluate --upgrade
# !python -m spacy download de_core_news_sm

In [2]:
# !pip uninstall -y torch torchtext torchvision torchaudio
# !pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 torchtext==0.18.0

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import torchtext
import torchtext.vocab # Import vocab directly
import numpy as np
import random
import spacy
import datasets
import tqdm
import evaluate
torchtext.disable_torchtext_deprecation_warning() # disable warnings

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

## Load Dataset

In [ ]:
dataset = datasets.load_dataset("bentrevett/multi30k") # https://huggingface.co/datasets/bentrevett/multi30k  (Eng-German Translation)

dataset

type(dataset)

train_data, valid_data, test_data = (
    dataset['train'],
    dataset['validation'],
    dataset['test']
)

In [8]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.'}

## Tokenization

In [ ]:
# python -m spacy download en_core_web_sm
# python -m spacy download de_core_news_sm

en_nlp = spacy.load('en_core_web_sm') # for english
de_nlp = spacy.load('de_core_news_sm') # for german

string = "What a lovely day it is today!"

[token.text for token in en_nlp.tokenizer(string)]

def tokenize_data(text, en_nlp, de_nlp, max_length, sos_token, eos_token, lower=True):
    en_tokens = [token.text for token in en_nlp.tokenizer(text['en'])][:max_length]
    de_tokens = [token.text for token in de_nlp.tokenizer(text['de'])][:max_length]

    if lower:
        en_tokens = [token.lower() for token in en_tokens]
        de_tokens = [token.lower() for token in de_tokens]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    de_tokens = [sos_token] + de_tokens + [eos_token]

    return {'en_tokens': en_tokens, 'de_tokens': de_tokens}

In [10]:
max_length = 1_000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp" : en_nlp,
    "de_nlp" : de_nlp,
    "max_length" : max_length,
    "lower" : lower,
    "sos_token" : sos_token,
    "eos_token" : eos_token
}

train_data = train_data.map(tokenize_data, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(tokenize_data, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_data, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}

## Building Vocab

In [ ]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

en_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

de_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["de_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

In [13]:
# get_itos, (itos = " int to string ")
en_vocab.get_itos()[:20]

['<unk>',
 '<pad>',
 '<sos>',
 '<eos>',
 'a',
 '.',
 'in',
 'the',
 'on',
 'man',
 'is',
 'and',
 'of',
 'with',
 'woman',
 ',',
 'two',
 'are',
 'to',
 'people']

In [14]:
de_vocab.get_itos()[:20]

['<unk>',
 '<pad>',
 '<sos>',
 '<eos>',
 '.',
 'ein',
 'einem',
 'in',
 'eine',
 ',',
 'und',
 'mit',
 'auf',
 'mann',
 'einer',
 'der',
 'frau',
 'die',
 'zwei',
 'einen']

In [15]:
# get_stoi (stoi = " string to int ")
print(en_vocab.get_stoi()["the"])
print(en_vocab.get_stoi()["man"])

len(en_vocab), len(de_vocab)

7
9


(5893, 7853)

In [16]:
assert en_vocab[unk_token] == de_vocab[unk_token]
assert en_vocab[pad_token] == de_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

In [17]:
# Using the set_default_index method we can set what value is returned when we try and get the index of a token outside of our vocabulary.
# In this case, the index of the unknown token, <unk>.

en_vocab.set_default_index(unk_index)
de_vocab.set_default_index(unk_index)

In [18]:
en_vocab["The"]

0

## Text to Sequence

In [ ]:
# tokens to sequence
tokens = ["i", "love", "watching", "action", "movies"]

seq = en_vocab.lookup_indices(tokens)
print(seq)

[956, 2169, 173, 1260, 0]


In [20]:
# sequence to tokens

en_vocab.lookup_tokens(seq)

['i', 'love', 'watching', 'action', '<unk>']

In [21]:
def text_to_seq(text, en_vocab, de_vocab):
    en_ids = en_vocab.lookup_indices(text["en_tokens"])
    de_ids = de_vocab.lookup_indices(text["de_tokens"])

    return {'en_ids': en_ids, 'de_ids': de_ids}

fun_kwargs = {
    "en_vocab" : en_vocab,
    "de_vocab" : de_vocab,
}

In [22]:
train_data = train_data.map(text_to_seq, fn_kwargs=fun_kwargs)
valid_data = valid_data.map(text_to_seq, fn_kwargs=fun_kwargs)
test_data = test_data.map(text_to_seq, fn_kwargs=fun_kwargs)

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [23]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>'],
 'en_ids': [2, 16, 24, 15, 25, 778, 17, 57, 80, 202, 1312, 5, 3],
 'de_ids': [2, 18, 26, 253, 30, 84, 20, 88, 7, 15, 110, 7647, 3171, 4, 3]}

In [24]:
en_vocab.lookup_tokens(train_data[0]["en_ids"])

['<sos>',
 'two',
 'young',
 ',',
 'white',
 'males',
 'are',
 'outside',
 'near',
 'many',
 'bushes',
 '.',
 '<eos>']

In [25]:
data_type = "torch"
format_columns = ["en_ids", "de_ids"]

train_data = train_data.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

In [26]:
train_data[0]

{'en_ids': tensor([   2,   16,   24,   15,   25,  778,   17,   57,   80,  202, 1312,    5,
            3]),
 'de_ids': tensor([   2,   18,   26,  253,   30,   84,   20,   88,    7,   15,  110, 7647,
         3171,    4,    3]),
 'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}

## Pad Sequences

In [ ]:
def get_collate_fun(pad_index):
    
    def collate_fun(batch):
        en_ids = [x["en_ids"] for x in batch]
        de_ids = [x["de_ids"] for x in batch]

        batch_en_ids = nn.utils.rnn.pad_sequence(
            en_ids,
            padding_value=pad_index,
        )
        batch_de_ids = nn.utils.rnn.pad_sequence(
            de_ids,
            padding_value=pad_index,
        )
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fun


## Data Loader

In [ ]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fun = get_collate_fun(pad_index)
    data_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        collate_fn=collate_fun,
    )
    return data_loader


In [29]:
batch_size = 128

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

## Building Model

### Encoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Input : [X, batch_size]

        embedded = self.dropout(self.embedding(x)) 
        # [X, batch_size, embedding_dim]

        output, (hidden, cell) = self.rnn(embedded)
    	# output = [X, batch_size, hidden_dim * d]            # d : direction : 1 = unidirectional, 2 = bidirectional
    	# hidden = [n_layers * d, batch_size, hidden_dim]     # If n_layers=2 and n_directions=2 (bidirectional), suppose we have 4 separate hidden states: layer0-forward, layer0-backward, layer1-forward, layer1-backward → hence 2 * 2 = 4.
    	# cell = [n_layers * d, batch_szie, hidden_dim]       

        return hidden, cell # Context vector for decoder

### Decoder

In [31]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim # [size of vocab]
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # Embedding
        self.embedding = nn.Embedding(output_dim, embedding_dim)

        # LSTM layer
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)

        # Linear layer
        self.fc_out = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # Input : [batch_size]   ->  single token at current time step
        # hidden : [n_layers, batch_size, hidden_dim]
        # cell : [n_layers, batch_size, hidden_dim]

        input = input.unsqueeze(0) 
        # Input : [1, batch_size]   # (RNN expects [seq_len, batch, features])
    
        embedded = self.dropout(self.embedding(input))
        # [1, batch_size, embedding_dim]

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))   # decoder start from previous hidden state and cell state of the encoder 
        # output : [1, batch_size, hidden_dim]
        # hidden = [n_layers, batch_size, hidden_dim]
        # cell = [n_layers, batch_size, hidden_dim]

        prediction = self.fc_out(output.squeeze(0)) 
        # (1, batch_size, hidden_dim) --> [batch_size, hidden_dim]
        # fc_out maps this to [batch_size, output_dim] → scores for each word in vocab


        return prediction, hidden, cell


In [32]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
                encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
                encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"


    def forward(self, src, trg, teacher_forcing_ratio):

        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # We need to start the decoder with the <sos> -> (start of sequence) token.
        # The first input will be the <sos> tokens.
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        # stores the output of decoder [1, batch_size, output_dim]
        outputs = torch.zeros(trg.shape[0], batch_size, trg_vocab_size).to(src.device) 

        # encoder_outputs is where we store all the hidden states from the top layer of the encoder
        # hidden and cell are the final hidden and cell states from the encoder
        # shape : [n_layer, batch_size, hidden_dim]
        hidden, cell = self.encoder(src)

        # fisrt input : <sos> for all sequences in batch
        input = trg[0, :]


        for t in range(1, trg_len):

            # input : ( <sos>, previous hidden & cell state) --> output : [batch_size, trg_vocab_size]
            # output : [batch_size, output_dim]
            # hidden, cell : [n_layers, batch_size, hidden_dim]
            output, hidden, cell = self.decoder(input, hidden, cell)

            # saves decoder output at step t
            outputs[t] = output

            # without teacher forcing : input = output.argmax(1)  # this leads exposure bias
            # with teacher forcing : input = trg[t-1]
            use_teacher_forcing = random.random() < teacher_forcing_ratio
        
            if use_teacher_forcing:
                input = trg[t] # use true token
            else:
                input = output.argmax(1) # use previously pridicted token

        return outputs # [trg_len, batch_size, trg_vocab_size]


### Model Initialization

In [ ]:
input_dim = len(de_vocab)
output_dim = len(en_vocab)
embedding_dim = 256
hidden_dim = 512
n_layers = 2
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(input_dim, embedding_dim, hidden_dim, n_layers, dropout)
decoder = Decoder(output_dim, embedding_dim, hidden_dim, n_layers, dropout)

model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
def init_weights(x):
    for name, params in x.named_parameters():
        nn.init.uniform_(params.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [35]:
def count_params(model):
    params = sum(param.numel() for param in model.parameters() if param.requires_grad)
    return params

In [36]:
print(f"The model has {count_params(model):,} trainable parameters")

The model has 13,898,501 trainable parameters


In [37]:
criterian = nn.CrossEntropyLoss(ignore_index=pad_index)
optimizer = optim.Adam(model.parameters())

## Training Loop

In [ ]:
def train_model(model, data_loader, criterian, optimizer, clip, teacher_forcing_ratio, device):
    model.train()
    epoch_loss = 0

    for i, batch in enumerate(train_data_loader):

        src = batch["de_ids"].to(device) # [src_len, batch_size]
        trg = batch["en_ids"].to(device) # [trg_len, batch_size]

        optimizer.zero_grad()

        output = model(src, trg, teacher_forcing_ratio) 
        # [trg_length, batch_size, trg vocab size]


        output_dim = output.shape[-1] 
        output = output[1:].view(-1, output_dim) 
        # output[1:] = exclude <sos> = [tag_len, batch_size, trg_vocab_size]
        # output = [(trg_len - 1) * batch_size, trg_vocab_size]
        # PyTorch CrossEntropyLoss expects input shape [N, C] (batch x classes).


        trg = trg[1:].view(-1)
        # exclude <sos>
        # trg = [(trg_len - 1) * batch_size]
        # each element = index of correct word in vocab

        loss = criterian(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        # If this total norm exceeds a specified max_norm, the gradients are scaled down proportionally so that their total norm equals max_norm

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(data_loader)


## Evaluation Loop

In [ ]:
#  however we must ensure we turn teacher forcing off for evaluation. This will cause the model to only use it's own predictions to make further predictions within a sentence
def evaluate_model(model, data_loader, criterian, device):
    model.eval()

    epoch_loss = 0
    with torch.no_grad():

        for i, batch in enumerate(valid_data_loader):

            src = batch["de_ids"].to(device)
            trg = batch["en_ids"].to(device)

            output = model(src, trg, 0) # turn off teacher forcing

            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg_len - 1) * batch_size, trg_vocab_size]

            trg = trg[1:].view(-1)
            # trg = [(trg_len - 1) * batch_size]

            loss = criterian(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(data_loader)

## Training

In [ ]:
num_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(num_epochs)):

    train_loss = train_model(
        model, 
        train_data_loader, 
        criterian, 
        optimizer, 
        clip, 
        teacher_forcing_ratio, 
        device,
    )

    valid_loss = evaluate_model(
        model, 
        valid_data_loader, 
        criterian, 
        device,
    )

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "translation-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

 10%|█         | 1/10 [01:01<09:09, 61.02s/it]

	Train Loss:   5.035 | Train PPL: 153.687
	Valid Loss:   4.961 | Valid PPL: 142.675


 20%|██        | 2/10 [02:02<08:10, 61.34s/it]

	Train Loss:   4.401 | Train PPL:  81.531
	Valid Loss:   4.682 | Valid PPL: 108.005


 30%|███       | 3/10 [03:06<07:18, 62.61s/it]

...

	Train Loss:   3.226 | Train PPL:  25.180
	Valid Loss:   3.956 | Valid PPL:  52.234


 90%|█████████ | 9/10 [09:15<01:01, 61.09s/it]

	Train Loss:   3.087 | Train PPL:  21.909
	Valid Loss:   3.895 | Valid PPL:  49.152


100%|██████████| 10/10 [10:16<00:00, 61.63s/it]

	Train Loss:   2.954 | Train PPL:  19.191
	Valid Loss:   3.735 | Valid PPL:  41.869


## Evaluating Model

In [41]:
model.load_state_dict(torch.load("translation-model.pt"))

test_loss = evaluate_model(model, test_data_loader, criterian, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

| Test Loss: 3.735 | Test PPL:  41.869 |


In [53]:
def translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        
        if isinstance(sentence, str):
            tokens = [token.text for token in de_nlp.tokenizer(sentence)]
        else:
            tokens = [token for token in sentence]
        
        if lower:
            tokens = [token.lower() for token in tokens]
        
        tokens = [sos_token] + tokens + [eos_token]
        ids = de_vocab.lookup_indices(tokens)
        
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        
        hidden, cell = model.encoder(tensor)
        
        inputs = en_vocab.lookup_indices([sos_token])
        
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            
            if predicted_token == en_vocab[eos_token]:
                break
            
        tokens = en_vocab.lookup_tokens(inputs)
    
    return tokens

In [54]:
sentence = test_data[0]["de"]
expected_translation = test_data[0]["en"]

sentence, expected_translation

('Ein Mann mit einem orangefarbenen Hut, der etwas anstarrt.',
 'A man in an orange hat starring at something.')

In [55]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

In [56]:
translation

['<sos>', 'a', 'man', 'in', 'a', 'black', 'jacket', 'is', 'a', '.', '<eos>']

In [57]:
sentence = "Ein Mann sitzt auf einer Bank."

In [58]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

In [59]:
translation

['<sos>', 'a', 'man', 'sitting', 'on', 'a', 'bench', '.', '<eos>']